## Using fbprophet to predict the Future of Kenya's Agriculture

In [1]:
import warnings; 
warnings.simplefilter('ignore')

In [4]:
from fbprophet import Prophet

ModuleNotFoundError: No module named 'fbprophet'

In [ ]:
import pandas as pd
df=pd.read_csv("S:\code\Kenya-s-Maize-Output-Time-Series\FAOSTAT_data_en_3-26-2023.csv")
df.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (CPC),Item,Year Code,Year,Unit,Value,Flag,Flag Description
0,QCL,Crops and livestock products,404,Kenya,5510,Production,01929.07,"Abaca, manila hemp, raw",1976,1976,tonnes,10.0,E,Estimated value
1,QCL,Crops and livestock products,404,Kenya,5510,Production,01929.07,"Abaca, manila hemp, raw",1977,1977,tonnes,10.0,E,Estimated value
2,QCL,Crops and livestock products,404,Kenya,5510,Production,01929.07,"Abaca, manila hemp, raw",1978,1978,tonnes,10.0,E,Estimated value
3,QCL,Crops and livestock products,404,Kenya,5510,Production,01929.07,"Abaca, manila hemp, raw",1979,1979,tonnes,10.0,E,Estimated value
4,QCL,Crops and livestock products,404,Kenya,5510,Production,01929.07,"Abaca, manila hemp, raw",1980,1980,tonnes,10.0,E,Estimated value


In [ ]:
df2=pd.DataFrame()    

year=[]
value=[]
count=0
for i in df['Item']:
    if i == 'Maize (corn)':
        year.append(df.iloc[count,9])
        value.append(df.iloc[count,11])

    count+=1


print(year,value)

[1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 200

In [ ]:
df2['Year']=year
df2['Produced']=value

In [ ]:
df2.head()

,Year,Produced
0,1961,750000.0
1,1962,900000.0
2,1963,1000000.0
3,1964,1050000.0
4,1965,1214000.0


In [ ]:
m = Prophet(interval_width=0.95, daily_seasonality=True)
model = m.fit(df2)

In [ ]:
future = m.make_future_dataframe(periods=100,freq='D')
forecast = m.predict(future)
forecast.head()

In [ ]:
plot1 = m.plot(forecast)
